# Section: Securing Federated Learning

- Lesson 1: Trusted Aggregator
- Lesson 2: Intro to Additive Secret Sharing
- Lesson 3: Intro to Fixed Precision Encoding
- Lesson 4: Secret Sharing + Fixed Precision in PySyft
- Final Project: Federated Learning wtih Encrypted Gradient Aggregation

# Lesson: Federated Learning with a Trusted Aggregator

In the last section, we learned how to train a model on a distributed dataset using Federated Learning. In particular, the last project aggregated gradients directly from one data owner to another. 

However, while in some cases it could be ideal to do this, what would be even better is to be able to choose a neutral third party to perform the aggregation.

As it turns out, we can use the same tools we used previously to accomplish this.

# Project: Federated Learning with a Trusted Aggregator

### Step 1: Create Data Owners

In [3]:
!pip install syft

     |████████████████████████████████| 225kB 2.4MB/s 
     |████████████████████████████████| 450kB 37.6MB/s 
     |████████████████████████████████| 204kB 44.0MB/s 
     |████████████████████████████████| 1.4MB 40.1MB/s 
     |████████████████████████████████| 256kB 38.6MB/s 
     |████████████████████████████████| 81kB 20.8MB/s 
     |████████████████████████████████| 389kB 42.1MB/s 
     |████████████████████████████████| 51kB 18.0MB/s 
     |████████████████████████████████| 276kB 47.1MB/s 
     |████████████████████████████████| 122kB 43.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/ad/9a/f4/3105b5209674ac77fcca7fede95184c62a95df0196888e0e76
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built zstd pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Found existing installation: msgpack 0.5.6
    Uninstalling msgpack-0.5.6:
     

In [0]:
import syft as sy
import torch as th
from torch import nn, optim

hook = sy.TorchHook(th)

W0710 00:12:36.633856 140373080033152 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0710 00:12:36.654864 140373080033152 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

# this step is important in the real-world application.
# You need to inform the workers of others existance
# you will probably have an ssh or http worker, not a virtual worker.

bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

W0709 17:50:10.055624 139991495161728 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0709 17:50:10.060528 139991495161728 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0709 17:50:10.061929 139991495161728 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0709 17:50:10.065222 139991495161728 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0709 17:50:10.066277 139991495161728 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0709 17:50:10.067300 139991495161728 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:secure_worker #objects:0>

In [0]:
# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)


### Step 2: Create the Model

In [0]:
# Iniitalize A Toy Model
model = nn.Linear(2,1)

### Step 3: Send a Copy of The Model to Alice and Bob

In [0]:
bobs_model = model.copy().send(bob) 
alices_model = model.copy().send(alice) 

bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)


### Step 4: Train Bob's and Alice's Models (in parallel)

In [0]:
for i in range(10):

    # Train Bob's Model
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target)**2).mean()
    bobs_loss.backward()

    bobs_opt.step()
    bobs_loss = bobs_loss.get().data
    print(f'Bob loss {bobs_loss}')

    # Train Alice's Model
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_pred - alices_target)**2).mean()
    alices_loss.backward()

    alices_opt.step()
    alices_loss = alices_loss.get().data
    print(f'alice loss {alices_loss}')


### Step 5: Send Both Updated Models to a Secure Worker

In [0]:
alices_model.move(secure_worker) # the move function iterates on all objects in alice and call send on it. NOTEICE: inline function
bobs_model.move(secure_worker)


In [0]:
secure_worker._objects # can you identify those objects in the Secure worker?

{21963037481: Parameter containing:
 tensor([[ 0.5020, -0.1802]], requires_grad=True),
 33860880968: Parameter containing:
 tensor([-0.1180], requires_grad=True),
 88005463002: Parameter containing:
 tensor([[ 0.5020, -0.1802]], requires_grad=True),
 95174969346: Parameter containing:
 tensor([-0.1180], requires_grad=True)}

### Step 6: Average The Models

In [0]:
# A pytorch trick: to set the weights: you have to either access the data[idx] 
# or wrap the set function with (with torch.no_grad():)
# below we illustrate the first way
# the next cell demonstrates the complete way:

model.weight.data[0].set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
model.bias.data[0].set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())

tensor([-0.1180])

### Step 7: The complete solution over multiple training sessions

In [0]:
iterations = 10
worker_iters = 5

for a_iter in range(iterations):

    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
    alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)

    for wi in range(worker_iters):
        # Train Bob's Model
        bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target) ** 2).sum()
        bobs_loss.backward()

        bobs_opt.step()
        bobs_loss = bobs_loss.get().data

        # Train Alice's Model
        alices_opt.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target) ** 2).sum()
        alices_loss.backward()

        alices_opt.step()
        alices_loss = alices_loss.get().data

    alices_model.move(secure_worker)
    bobs_model.move(secure_worker)

    with th.no_grad():

        model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
        model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(0.0002) Alice:tensor(5.8867e-06)
Bob:tensor(0.0002) Alice:tensor(4.2680e-06)
Bob:tensor(0.0001) Alice:tensor(3.1370e-06)
Bob:tensor(0.0001) Alice:tensor(2.3311e-06)
Bob:tensor(8.0358e-05) Alice:tensor(1.7466e-06)
Bob:tensor(6.2014e-05) Alice:tensor(1.3172e-06)
Bob:tensor(4.7825e-05) Alice:tensor(9.9825e-07)
Bob:tensor(3.6863e-05) Alice:tensor(7.5948e-07)
Bob:tensor(2.8402e-05) Alice:tensor(5.7938e-07)
Bob:tensor(2.1877e-05) Alice:tensor(4.4302e-07)


In [0]:
preds = model(data)
loss = ((preds - target) ** 2).mean()

In [0]:
print(preds)
print(target)
print(loss.data)

tensor([[0.0131],
        [0.0107],
        [0.9865],
        [0.9841]], grad_fn=<AddmmBackward>)
tensor([[0.],
        [0.],
        [1.],
        [1.]], requires_grad=True)
tensor(0.0002)


# Lesson: Intro to Additive Secret Sharing

While being able to have a trusted third party to perform the aggregation is certainly nice, in an ideal setting we wouldn't have to trust anyone at all. This is where Cryptography can provide an interesting alterantive. 

Specifically, we're going to be looking at a simple protocol for Secure Multi-Party Computation called Additive Secret Sharing. This protocol will allow multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation. In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.

Let's start by considering the number 5, which we'll put into a varible x

In [0]:
x = 5

Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [0]:
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

5

Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted). 

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [0]:
bob_x_share *= 2
alice_x_share *= 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

10

This even works for addition between two shared values!!

In [0]:
# encrypted "5"
bob_x_share = 2
alice_x_share = 3

# encrypted "7"
bob_y_share = 5
alice_y_share = 2

# encrypted 5 + 7
bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
decrypted_z

12

As you can see, we just added two numbers together while they were still encrypted ( you are the dealer, thus still can see --and know everything-- but Alice and Bob did not really know anything about the overall deal --the sum of the numbers; the secrete--)

In this case, however, to keep the secrete is not about the relaiability / integrity of the person who holds it, rather the fact that those shareholders do not know each other!!!

One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.

This would be quite bad, but can be solved through a simple fix. Decryption happens by summing all the shares together MODULUS some constant.

In [0]:
x = 5

Q = 9923 # the little fix; a very large prime number

bob_x_share = 1525 # give bob a random number.. try with negatives

alice_x_share = Q - bob_x_share + x

print(f'Bob share= {bob_x_share}')
print(f'Alice share= {alice_x_share}')
print(f'Add both shares= {bob_x_share + alice_x_share}... Wrong secrete')

Bob share= 1525
Alice share= 8403
Add both shares= 9928... Wrong secrete


In [0]:
# Decryption happens by summing all the shares together MODULUS some constant

(bob_x_share + alice_x_share) % Q

5

So now, as you can see, both shares are wildly larger than the number being shared, meaning that individual shares no longer leak this inforation. However, all the properties we discussed earlier still hold! (addition, encryption, decryption, etc.)

# Project: Build Methods for Encrypt, Decrypt, and Add 

In this project, you must take the lessons we learned in the last section and write general methods for encrypt, decrypt, and add. Store shares for a variable in a tuple like so:  `x_share = (2,5,7)`

Even though normally those shares would be distributed among several workers, you can store them in ordered tuples like this for now.

In [0]:
import random

Q = 23740629843760239486723

# accepts a secrete and shares it among n_shahre, returns a tuple with n_share shares
def encrypt(x, n_share=3):
    
    shares = list()
    
    for i in range(n_share-1):
        shares.append(random.randint(0,Q))
        
    shares.append(Q - (sum(shares) % Q) + x)
    
    return tuple(shares)

  # accepts a tuple of shares and return the decrypted values
def decrypt(shares):
    return sum(shares) % Q

In [0]:
shares = encrypt(x=7, n_share=10)
shares

(13610489447565512710478,
 4741001210397115637137,
 18524098276181263782152,
 23009444748397071753564,
 13470784748890957219057,
 6165981231992267330792,
 5022544955327868230424,
 4911008412425769794252,
 21594905830452253817470,
 7652890357171117158296)

In [0]:
decrypt(shares)

7

In [0]:
def add(a, b):
    c = list()
    assert(len(a) == len(b))
    for i in range(len(a)):
        c.append((a[i] + b[i]) % Q)
    return tuple(c)

In [0]:
# run this block of code multiple times, what do you notice baout x and y?

x = encrypt(5)
print(x)
y = encrypt(7)
print(y)

z = add(x,y)
decrypt(z)

(9507620613546751793694, 5667183245941444293658, 8565825984272043399376)
(3846464344348558391559, 2106515060740304535225, 17787650438671376559946)


12

# Lesson: Intro to Fixed Precision Encoding

As you may remember, our goal is to aggregate gradients using this new Secret Sharing technique. However, the protocol we've just explored in the last section uses positive integers. However, our neural network weights are NOT integers. Instead, our weights are decimals (floating point numbers).

Not a huge deal! We just need to use a fixed precision encoding, which lets us do computation over decimal numbers using integers!

In [0]:
BASE= 10  # integer numbers (normal numbers)
PRECISION= 3  # how many porecisions after the decimal point to encode
Q = 23740629843760239486723 

In [0]:
def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION

In [0]:
encode(-0.600)

23740629843760239345664

In [0]:
decode(23740629843760239345664)

-141.059

In [0]:
# run this cell multiple times and notice the changes

x = encrypt(encode(5.5))
y = encrypt(encode(5.5))
z = add(x,y)


print(f'Value of x = {x}')
print(f'Value of y = {y}')
print(f'Value of sum = {z}')
print(f'decoded sum = {decode(decrypt(z))}')

Value of x = (13989725804496061343170, 5045709939432612913353, 4705194099831565235700)
Value of y = (4342207001814149424776, 472976683055563301440, 18925446158890526766007)
Value of sum = (18331932806310210767946, 5518686622488176214793, 23630640258722092001707)
decoded sum = 11.0


# Lesson: Secret Sharing + Fixed Precision in PySyft

While writing things from scratch is certainly educational, PySyft makes a great deal of this much easier for us through its abstractions.

In [0]:
# Its a good idea to restrat the environment at this point

import syft as sy
import torch as th
from torch import nn, optim

hook = sy.TorchHook(th)

# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")


bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

W0710 00:13:35.343144 140373080033152 hook.py:98] Torch was already hooked... skipping hooking process
W0710 00:13:35.347748 140373080033152 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0710 00:13:35.350229 140373080033152 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0710 00:13:35.363440 140373080033152 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0710 00:13:35.367980 140373080033152 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0710 00:13:35.373629 140373080033152 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0710 00:13:35.376579 140373080033152 base.py:628] Worker bob already exists. Replacing 

<VirtualWorker id:secure_worker #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]) ##### the following steps work for integer values only

### Secret Sharing Using PySyft

We can share using the simple` .share()` method!

In [0]:
x = x.share(bob, alice, secure_worker) # implemets additive secrete shares
x # it's a pointer to three secrete shares

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:90554420523 -> bob:20721516275]
	-> (Wrapper)>[PointerTensor | me:61476636065 -> alice:3222656444]
	-> (Wrapper)>[PointerTensor | me:31067325809 -> secure_worker:47245444006]
	*crypto provider: me*

In [0]:
bob._objects # take a look at one of the shares.. Large random numbers --encrypted

{20721516275: tensor([4035456453582792227,  767473370537760854, 1697678287077022869,
         4178212545784135116, 1773324359685915620])}

and as you can see, Bob now has one of the shares of x! Furthermore, we can still call addition in this state, and PySyft will automatically perform the remote execution for us!

In [0]:
y = x + x
y

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:76951768469 -> bob:29940855800]
	-> (Wrapper)>[PointerTensor | me:60802860878 -> alice:51080765001]
	-> (Wrapper)>[PointerTensor | me:30052547358 -> secure_worker:94278570382]
	*crypto provider: me*

In [0]:
bob._objects # bob has another tensor.. why?

{20721516275: tensor([4035456453582792227,  767473370537760854, 1697678287077022869,
         4178212545784135116, 1773324359685915620]),
 29940855800: tensor([8070912907165584454, 1534946741075521708, 3395356574154045738,
         8356425091568270232, 3546648719371831240])}

In [0]:
y.get() # notice that we have doubled the values on each worker!

tensor([ 2,  4,  6,  8, 10])


### Fixed Precision using PySyft

We can also convert a tensor to fixed precision using `.fix_precision()`
This is important when doing FL since the values you are sharing are decimal numbers.

**Q: what values are we sharing? what are the shared secretes?**

In [0]:
x = th.tensor([0.1, 0.2, 0.3, 0.4]) # a decimal tensor!

In [0]:
x

tensor([0.1000, 0.2000, 0.3000, 0.4000])

In [0]:
x = x.fix_prec() # call this function to encode the decimal tensor using fixed precision
x # notice that this is a tensor chain <-------

(Wrapper)>FixedPrecisionTensor>tensor([100, 200, 300, 400])

In [0]:
print(f'Type of x {type(x)}')
print(f'Type of x.child {type(x.child)}')
print(f'Type of x.child.child {type(x.child.child)}') # <-- this is how you get the data from a fixed precision tensor

Type of x <class 'syft.frameworks.torch.tensors.interpreters.native.Tensor'>
Type of x.child <class 'syft.frameworks.torch.tensors.interpreters.precision.FixedPrecisionTensor'>
Type of x.child.child <class 'syft.frameworks.torch.tensors.interpreters.native.Tensor'>


In [0]:
# to decode the previous tensor
print(x.float_prec())

tensor([0.1000, 0.2000, 0.3000, 0.4000])


In [0]:
y = x + x
y

(Wrapper)>FixedPrecisionTensor>tensor([200, 400, 600, 800])

In [0]:
y = y.float_prec()
y

tensor([0.2000, 0.4000, 0.6000, 0.8000])

### Shared Fixed Precision

And of course, we can combine the two!

In [0]:
x = th.tensor([0.1, 0.2, 0.3])

In [0]:
x = x.fix_prec().share(bob, alice)
x

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:15840963332 -> bob:80849880030]
	-> (Wrapper)>[PointerTensor | me:67003867711 -> alice:69489804107]
	*crypto provider: me*

In [0]:
bob._objects

{20721516275: tensor([4035456453582792227,  767473370537760854, 1697678287077022869,
         4178212545784135116, 1773324359685915620]),
 80849880030: tensor([ 584142369266789926, 4451511967337482695, 2221762689720435853])}

In [0]:
y = x + x

In [0]:
y.get().float_prec()

tensor([0.2000, 0.4000, 0.6000])

# Final Project: Federated Learning with Encrypted Gradient Aggregation

Reuse your project from the Secuered Agggregator above to train the same model using the FL appraoch, but this time use the Additive Sharing Encryption so that the participating members have access to their own models only. There is no secure aggregator in this senario. 
Include four members in this project (Alice, Bob, Ted, and Carol). 

Hint: we will *share* the model with the shareholders, do addition on the encrypted models, and then aggregate the final model locally. 

In [4]:
# create cells as needed. Separate your solution into logical cells.

import syft as sy
import torch as th
from torch import nn, optim

hook = sy.TorchHook(th)

W0711 21:33:40.544811 139848781096832 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0711 21:33:40.562182 139848781096832 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
# create a four workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
ted = sy.VirtualWorker(hook, id="ted")
carol = sy.VirtualWorker(hook, id="carol")
crypto_provider = sy.VirtualWorker(id="crypto_provider", hook=hook)


In [0]:
# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

In [9]:
# get pointers to training data on each worker by
# sending some training data to bob and alice
data = data.fix_precision().share(bob, alice, ted, carol, crypto_provider=crypto_provider,  requires_grad=True)
target = target.fix_precision().share(bob, alice, ted, carol, crypto_provider=crypto_provider,  requires_grad=True)
print(bob._objects)

{68459367522: tensor([[ 409015437152250994, 2163217370365550600],
        [3014282020116874072, 1169397004252231653],
        [3343771117972350882,   45064311311608256],
        [3492956924693620901, 1589670721898582923]]), 65064634744: tensor([[3664689524447765173],
        [2255753470245513401],
        [1566009422138110582],
        [4508293847029207664]])}


In [11]:
y = data + data
print(y.get().float_prec())

AttributeError: ignored

In [0]:
# Iniitalize A Toy Model
model = nn.Linear(2,1)

In [0]:
model = model.fix_precision().share(bob, alice, ted, carol, crypto_provider=crypto_provider,  requires_grad=True)
opt = optim.SGD(params=model.parameters(), lr=0.1).fix_precision()

In [14]:
print(model.weight)
print(opt)

Parameter>AutogradTensor>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:89278881538 -> bob:70699145725]
	-> (Wrapper)>[PointerTensor | me:79461477386 -> alice:33247571898]
	-> (Wrapper)>[PointerTensor | me:19709171026 -> ted:87970193773]
	-> (Wrapper)>[PointerTensor | me:91467082624 -> carol:62609026037]
	*crypto provider: crypto_provider*
SGD (
Parameter Group 0
    dampening: 0
    lr: FixedPrecisionTensor>tensor([100])
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [15]:
for i in range(10):

    # Train Model'S
    opt.zero_grad()
    pred = model(data)
    loss = ((pred - target)**2).sum()
    loss.backward()

    opt.step()
    print(F'loss {loss.get().float_precision()}')

loss tensor([-1.8447e+13])
loss tensor([3.3322e+13])
loss tensor([-2.7334e+13])
loss tensor([1.9980e+11])
loss tensor([1.5769e+13])
loss tensor([-2.7742e+13])
loss tensor([-2.3631e+13])
loss tensor([-3.0581e+10])
loss tensor([-2.9130e+13])
loss tensor([-2.4908e+13])


In [16]:
#models in the workers
model.weight

Parameter containing:
Parameter>AutogradTensor>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:83882096920 -> bob:82835065163]
	-> (Wrapper)>[PointerTensor | me:81156891744 -> alice:95407101871]
	-> (Wrapper)>[PointerTensor | me:83637260571 -> ted:38023451404]
	-> (Wrapper)>[PointerTensor | me:47241347915 -> carol:74614703756]
	*crypto provider: crypto_provider*

In [17]:
#aggregated model 
model = model.get()
print(model.float_precision().weight)

Parameter containing:
tensor([[-2.0017e+12, -3.9166e+12]], requires_grad=True)
